In [0]:
help(spark.udf.register)

Help on method register in module pyspark.sql.udf:

register(name, f, returnType=None) method of pyspark.sql.udf.UDFRegistration instance
 Register a Python function (including lambda function) or a user-defined function
 as a SQL function.
 
 .. versionadded:: 1.3.1
 
 Parameters
 ----------
 name : str,
 name of the user-defined function in SQL statements.
 f : function, :meth:`pyspark.sql.functions.udf` or :meth:`pyspark.sql.functions.pandas_udf`
 a Python function, or a user-defined function. The user-defined function can
 be either row-at-a-time or vectorized. See :meth:`pyspark.sql.functions.udf` and
 :meth:`pyspark.sql.functions.pandas_udf`.
 returnType : :class:`pyspark.sql.types.DataType` or str, optional
 the return type of the registered user-defined function. The value can
 be either a :class:`pyspark.sql.types.DataType` object or a DDL-formatted type string.
 `returnType` can be optionally specified when `f` is a Python function but not
 when `f` is a user-defined function. Please see the examples below.
 
 Returns
 -------
 function
 a user-defined function
 
 Notes
 -----
 To register a nondeterministic Python function, users need to first build
 a nondeterministic user-defined function for the Python function and then register it
 as a SQL function.
 
 Examples
 --------
 1. When `f` is a Python function:
 
 `returnType` defaults to string type and can be optionally specified. The produced
 object must match the specified type. In this case, this API works as if
 `register(name, f, returnType=StringType())`.
 
 >>> strlen = spark.udf.register("stringLengthString", lambda x: len(x))
 >>> spark.sql("SELECT stringLengthString('test')").collect()
 [Row(stringLengthString(test)='4')]
 
 >>> spark.sql("SELECT 'foo' AS text").select(strlen("text")).collect()
 [Row(stringLengthString(text)='3')]
 
 >>> from pyspark.sql.types import IntegerType
 >>> _ = spark.udf.register("stringLengthInt", lambda x: len(x), IntegerType())
 >>> spark.sql("SELECT stringLengthInt('test')").collect()
 [Row(stringLengthInt(test)=4)]
 
 >>> from pyspark.sql.types import IntegerType
 >>> _ = spark.udf.register("stringLengthInt", lambda x: len(x), IntegerType())
 >>> spark.sql("SELECT stringLengthInt('test')").collect()
 [Row(stringLengthInt(test)=4)]
 
 2. When `f` is a user-defined function (from Spark 2.3.0):
 
 Spark uses the return type of the given user-defined function as the return type of
 the registered user-defined function. `returnType` should not be specified.
 In this case, this API works as if `register(name, f)`.
 
 >>> from pyspark.sql.types import IntegerType
 >>> from pyspark.sql.functions import udf
 >>> slen = udf(lambda s: len(s), IntegerType())
 >>> _ = spark.udf.register("slen", slen)
 >>> spark.sql("SELECT slen('test')").collect()
 [Row(slen(test)=4)]
 
 >>> import random
 >>> from pyspark.sql.functions import udf
 >>> from pyspark.sql.types import IntegerType
 >>> random_udf = udf(lambda: random.randint(0, 100), IntegerType()).asNondeterministic()
 >>> new_random_udf = spark.udf.register("random_udf", random_udf)
 >>> spark.sql("SELECT random_udf()").collect() # doctest: +SKIP
 [Row(random_udf()=82)]
 
 >>> import pandas as pd # doctest: +SKIP
 >>> from pyspark.sql.functions import pandas_udf
 >>> @pandas_udf("integer") # doctest: +SKIP
 ... def add_one(s: pd.Series) -> pd.Series:
 ... return s + 1
 ...
 >>> _ = spark.udf.register("add_one", add_one) # doctest: +SKIP
 >>> spark.sql("SELECT add_one(id) FROM range(3)").collect() # doctest: +SKIP
 [Row(add_one(id)=1), Row(add_one(id)=2), Row(add_one(id)=3)]
 
 >>> @pandas_udf("integer") # doctest: +SKIP
 ... def sum_udf(v: pd.Series) -> int:
 ... return v.sum()
 ...
 >>> _ = spark.udf.register("sum_udf", sum_udf) # doctest: +SKIP
 >>> q = "SELECT sum_udf(v1) FROM VALUES (3, 0), (2, 0), (1, 1) tbl(v1, v2) GROUP BY v2"
 >>> spark.sql(q).collect() # doctest: +SKIP
 [Row(sum_udf(v1)=1), Row(sum_udf(v1)=5)]

In [0]:
df = spark.read.json('/public/retail_db_json/orders')

In [0]:
df.show()

+-----------------+--------------------+--------+---------------+
order_customer_id| order_date|order_id| order_status|
+-----------------+--------------------+--------+---------------+
 11599|2013-07-25 00:00:...| 1| CLOSED|
 256|2013-07-25 00:00:...| 2|PENDING_PAYMENT|
 12111|2013-07-25 00:00:...| 3| COMPLETE|
 8827|2013-07-25 00:00:...| 4| CLOSED|
 11318|2013-07-25 00:00:...| 5| COMPLETE|
 7130|2013-07-25 00:00:...| 6| COMPLETE|
 4530|2013-07-25 00:00:...| 7| COMPLETE|
 2911|2013-07-25 00:00:...| 8| PROCESSING|
 5657|2013-07-25 00:00:...| 9|PENDING_PAYMENT|
 5648|2013-07-25 00:00:...| 10|PENDING_PAYMENT|
 918|2013-07-25 00:00:...| 11| PAYMENT_REVIEW|
 1837|2013-07-25 00:00:...| 12| CLOSED|
 9149|2013-07-25 00:00:...| 13|PENDING_PAYMENT|
 9842|2013-07-25 00:00:...| 14| PROCESSING|
 2568|2013-07-25 00:00:...| 15| COMPLETE|
 7276|2013-07-25 00:00:...| 16|PENDING_PAYMENT|
 2667|2013-07-25 00:00:...| 17| COMPLETE|
 1205|2013-07-25 00:00:...| 18| CLOSED|
 9488|2013-07-25 00:00:...| 19|PENDING_PAYMENT|
 9198|2013-07-25 00:00:...| 20| PROCESSING|
+-----------------+--------------------+--------+---------------+
only showing top 20 rows

In [0]:
dc = spark.udf.register('date_convert', lambda d: int(d[:10].replace('-', '')))

In [0]:
dc

Out[4]: <function __main__.<lambda>(d)>

In [0]:
df.select(dc('order_date').alias('order_date')).show()

+----------+
order_date|
+----------+
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
 20130725|
+----------+
only showing top 20 rows

In [0]:
df.filter(dc('order_date') == 20140101).show()

+-----------------+--------------------+--------+---------------+
order_customer_id| order_date|order_id| order_status|
+-----------------+--------------------+--------+---------------+
 3414|2014-01-01 00:00:...| 25876|PENDING_PAYMENT|
 5549|2014-01-01 00:00:...| 25877|PENDING_PAYMENT|
 9084|2014-01-01 00:00:...| 25878| PENDING|
 5118|2014-01-01 00:00:...| 25879| PENDING|
 10146|2014-01-01 00:00:...| 25880| CANCELED|
 3205|2014-01-01 00:00:...| 25881|PENDING_PAYMENT|
 4598|2014-01-01 00:00:...| 25882| COMPLETE|
 11764|2014-01-01 00:00:...| 25883| PENDING|
 7904|2014-01-01 00:00:...| 25884|PENDING_PAYMENT|
 7253|2014-01-01 00:00:...| 25885| PENDING|
 8195|2014-01-01 00:00:...| 25886| PROCESSING|
 10062|2014-01-01 00:00:...| 25887| PENDING|
 6735|2014-01-01 00:00:...| 25888| COMPLETE|
 10045|2014-01-01 00:00:...| 25889| COMPLETE|
 2581|2014-01-01 00:00:...| 25890| PENDING|
 3037|2014-01-01 00:00:...| 25891| CLOSED|
 3853|2014-01-01 00:00:...| 25892| ON_HOLD|
 8679|2014-01-01 00:00:...| 25893|PENDING_PAYMENT|
 7839|2014-01-01 00:00:...| 25894| PROCESSING|
 1044|2014-01-01 00:00:...| 25895| COMPLETE|
+-----------------+--------------------+--------+---------------+
only showing top 20 rows

In [0]:
df. \
    groupBy(dc('order_date').alias('order_date')). \
    count(). \
    withColumnRenamed('count', 'order_count'). \
    show()

+----------+-----------+
order_date|order_count|
+----------+-----------+
 20130919| 206|
 20140303| 266|
 20140202| 192|
 20140310| 235|
 20130809| 125|
 20130817| 253|
 20131015| 174|
 20140114| 209|
 20131029| 128|
 20140130| 254|
 20130824| 265|
 20130913| 103|
 20130914| 276|
 20130825| 200|
 20131031| 208|
 20140304| 257|
 20130731| 252|
 20130730| 227|
 20131116| 120|
 20131213| 135|
+----------+-----------+
only showing top 20 rows